In [10]:
import numpy as np #для записи градиента и проверки условия остановки
from scipy import optimize #для ондомерной оптимизации параметра a
import sympy as sp #библиотека для производных
from sympy import diff

In [20]:
x0, y0 = 0.9, -0.9 #начальная точка
x,y = sp.symbols('x,y') #обозначения переменных чтобы потом вычислить производные
eps = 0.01 #погрешность
def f(x,y):
   #ф-ция данная в задаче
  return 1.5*y**2 + 3*x**2 + 3*y - 6*x + 1


In [21]:
def grad(f, a, b): #ф-ция вычисления градиента
  dx = diff(f(x,y), x).subs(sympy.Symbol("x"), a).subs(sympy.Symbol("y"), b) #значение частной производной по иксу в точке
  dy = diff(f(x,y), y).subs(sympy.Symbol("y"), b).subs(sympy.Symbol("x"), a) #значение частной производной по игрику в точке
  return np.array([dx, dy], dtype=float)#вывод градиента


def find_alpha(f,x,y):
  #ф-ция поиска альфы для наискорейшего спука и первой итерации сопряженных градиентов
  df  = lambda a : f(x-a*grad(f,x,y)[0], y-a*grad(f,x,y)[1])
  return optimize.minimize_scalar(df).x# ф-ция одномерной оптимизации (поиска минимума) в scipy

def fast_fall(x,y,f,eps, k):
  #ф-ция наискорейшего спуска
  if max(abs(grad(f, x, y))) < eps: #условие выхода из рекурссии
    print("наискорейший спуск")
    print("Pont[x_k,y_k] (",x,y,")")
    print("Gradient", grad(f,x,y))
    print("число итераций", k)
    return
  alpha = find_alpha(f, x, y)
  fast_fall(x-alpha*grad(f,x,y)[0], y-alpha*grad(f,x,y)[1], f, eps, k+1)


def find_alpha_sg(f,x,y,b):
  #находим альфа в одномерной ф-ции используя "сопряженный градиент"
  df  = lambda a : f(x-a*b[0], y-a*b[1])
  return optimize.minimize_scalar(df).x


def sopr_grad(x,y,f,eps, k, new_grad):
  #ф-ция сопряженных градиентов
  if np.linalg.norm(grad(f, x, y)) < eps:
    print("сопряженные градиенты")
    print("Pont[x_k,y_k] (",x,y,")")
    print("Gradient", grad(f,x,y))
    print("число итераций", k)
    return
  if k == 0:# первая итерация как в методе наискорейшего спуска
    alpha = find_alpha(f, x, y)
    sopr_grad(x-alpha*grad(f,x,y)[0], y-alpha*grad(f,x,y)[1], f, eps, k+1, new_grad)
  else:
    del_grad = (np.linalg.norm(grad(f,x,y))**2)/(np.linalg.norm(new_grad)**2)#отношение норм для нового вспомагательного вектора
    new_grad = grad(f,x,y)+del_grad*new_grad #новый вектор("сопряженный градиент") в направлении убывания
    alpha = find_alpha_sg(f, x, y, new_grad)
    sopr_grad(x-alpha*new_grad[0], y-alpha*new_grad[1], f, eps, k+1, grad(f,x,y))





In [22]:
fast_fall(0.9, -0.9, f, 0.01, 0) #запуск ф-ции наискорейшего спуска

наискорейший спуск
Pont[x_k,y_k] ( 1.000823040444604 -0.9967078264633902 )
Gradient [0.00493824 0.00987652]
число итераций 3


In [23]:
sopr_grad(0.9, -0.9, f, 0.01, 0, grad(f, 0.9, -0.9))#запуск ф-ции сопряженных градиентов

сопряженные градиенты
Pont[x_k,y_k] ( 1.000000009113279 -0.9999999898150752 )
Gradient [5.46796741e-08 3.05547743e-08]
число итераций 2
